In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *
from tabulate import tabulate
#from brokenaxes import brokenaxes
import zipfile
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

In [ ]:
# Circuit STR data
ASD_CircuitsSet = pd.read_csv(
    "/home/jw3514/Work/ASD_Circuits/notebooks/ASD.SA.Circuits.Size46.csv",
    index_col="idx")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
RealSibSTRs = pd.read_csv("../dat/Unionize_bias/sib.top61.Z2.csv", 
                          index_col="STR").head(46).index.values

In [ ]:
adj_mat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/WeightMat.Ipsi.csv", index_col=0)
dist_mat = pd.read_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.ipsi.csv", index_col=0)
dist_mat.columns = dist_mat.index.values

In [ ]:
def MaskDistMat(Mat1, Mat2, cutoff, m='lt'):
    New_Mat2 = Mat2.copy(deep=True)
    for STR_i in Mat1.index.values:
        for STR_j in Mat1.columns.values:
            if m == 'gt':
                if Mat1.loc[STR_i, STR_j] >= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
            elif m == "lt":
                if Mat1.loc[STR_i, STR_j] <= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
    return New_Mat2

def MaskDistMat_xx(distance_mat, Conn_mat, cutoff, cutoff2, keep='gt'):
    Conn_mat_new = Conn_mat.copy(deep=True)
    distance_mat_new = distance_mat.copy(deep=True)
    for STR_i in distance_mat.index.values:
        for STR_j in distance_mat.columns.values:
            if keep == 'gt':
                if distance_mat.loc[STR_i, STR_j] >= cutoff:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0
            elif keep == "lt":
                if distance_mat.loc[STR_i, STR_j] <= cutoff:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0   
            elif keep=="bw":
                if distance_mat.loc[STR_i, STR_j] >= cutoff and distance_mat.loc[STR_i, STR_j] <= cutoff2:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0   
    return Conn_mat_new, distance_mat_new

def MaskDistMat_xy(distance_mat, Conn_mat, cutoff, cutoff2, keep='gt'):
    Conn_mat_new = Conn_mat.copy(deep=True)
    distance_mat_new = distance_mat.copy(deep=True)
    for STR_i in distance_mat.index.values:
        for STR_j in distance_mat.columns.values:
            if STR_i == STR_j:
                Conn_mat_new.loc[STR_i, STR_j] = 0
                distance_mat_new.loc[STR_i, STR_j] = 0
                continue
            if keep == 'gt':
                if distance_mat.loc[STR_i, STR_j] >= cutoff:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0
            elif keep == "lt":
                if distance_mat.loc[STR_i, STR_j] <= cutoff:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0
            elif keep=="bw":
                if distance_mat.loc[STR_i, STR_j] >= cutoff and distance_mat.loc[STR_i, STR_j] <= cutoff2:
                    Conn_mat_new.loc[STR_i, STR_j] = Conn_mat.loc[STR_i, STR_j]
                    distance_mat_new.loc[STR_i, STR_j] = distance_mat.loc[STR_i, STR_j]
                else:
                    Conn_mat_new.loc[STR_i, STR_j] = 0
                    distance_mat_new.loc[STR_i, STR_j] = 0
    return Conn_mat_new, distance_mat_new

In [ ]:
Cartesian_distances_w_edge = MaskDistMat(adj_mat, dist_mat, cutoff=0)
#Distance_Cuts = [0, 1000, 2000, 3000, 4000, 5000, 100000]
Distance_Cuts = [0, 1000, 2000, 3000, 4000, 100000]

N_Connections_total = []
N_Pairs_total = []
Cutted_DistMat = {}
Cutted_AdjMat = {}
for i, cut in enumerate(Distance_Cuts[:-1]):
    Conn_mat_new, distance_mat_new = MaskDistMat_xy(dist_mat, adj_mat, keep="bw",
                                                cutoff=Distance_Cuts[i], cutoff2=Distance_Cuts[i+1])
    Cutted_DistMat[i] = distance_mat_new
    Cutted_AdjMat[i] = Conn_mat_new
    N_Connections_total.append(np.count_nonzero(Conn_mat_new))
    N_Pairs_total.append(np.count_nonzero(distance_mat_new))
N_Connections_total = np.array(N_Connections_total)
N_Pairs_total = np.array(N_Pairs_total)

In [ ]:
N_Connections_total, N_Pairs_total

In [ ]:
162 + 827 + 813 + 609 + 652

In [ ]:
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
ASD_Connections = []
ASD_Pairs = []
for i,v in enumerate(Distance_Cuts[:-1]):
    adj_mat_ = Cutted_AdjMat[i]
    dist_mat_ = Cutted_DistMat[i]
    adj_mat_asd = adj_mat_.loc[ASD_Circuits,ASD_Circuits]
    dist_mat_asd = dist_mat_.loc[ASD_Circuits,ASD_Circuits]
    Nconn = np.count_nonzero(adj_mat_asd)
    Npair = np.count_nonzero(dist_mat_asd)
    ASD_Connections.append(Nconn)
    ASD_Pairs.append(Npair)

ASD_Connections = np.array(ASD_Connections)
ASD_Pairs = np.array(ASD_Pairs)

ASD_Connections/ASD_Pairs

In [ ]:
ASD_Connections, ASD_Pairs

In [ ]:
SIB_Connections = []
SIB_Pairs = []
for i,v in enumerate(Distance_Cuts[:-1]):
    adj_mat_ = Cutted_AdjMat[i]
    dist_mat_ = Cutted_DistMat[i]
    adj_mat_sib = adj_mat_.loc[RealSibSTRs,RealSibSTRs]
    dist_mat_sib = dist_mat_.loc[RealSibSTRs,RealSibSTRs]
    Nconn = np.count_nonzero(adj_mat_sib)
    Npair = np.count_nonzero(dist_mat_sib)
    SIB_Connections.append(Nconn)
    SIB_Pairs.append(Npair)

SIB_Connections = np.array(SIB_Connections)
SIB_Pairs = np.array(SIB_Pairs)

SIB_Connections/SIB_Pairs

In [ ]:
SIB_Connections, SIB_Pairs

In [ ]:
subsib_connections = []
subsib_Pairs = []
Sim_dir = "../dat/Unionize_bias/SubSampleSib//"
for file in os.listdir(Sim_dir):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(Sim_dir+file, index_col="STR")
    _SibSTRs = df.head(46).index.values
    _SIB_Connections = []
    _SIB_Pairs = []
    for i,v in enumerate(Distance_Cuts[:-1]):
        adj_mat_ = Cutted_AdjMat[i]
        dist_mat_ = Cutted_DistMat[i]
        adj_mat_sib = adj_mat_.loc[_SibSTRs,_SibSTRs]
        dist_mat_sib = dist_mat_.loc[_SibSTRs,_SibSTRs]
        Nconn = np.count_nonzero(adj_mat_sib)
        Npair = np.count_nonzero(dist_mat_sib)
        _SIB_Connections.append(Nconn)
        _SIB_Pairs.append(Npair)
    subsib_connections.append(_SIB_Connections)
    subsib_Pairs.append(_SIB_Pairs)
subsib_connections = np.array(subsib_connections)
subsib_Pairs = np.array(subsib_Pairs)

In [ ]:
subsib_connections.shape

In [ ]:
subsib_conns_mean =  np.mean(subsib_connections, axis=0)
subsib_conns_std =  np.std(subsib_connections, axis=0)
subsib_pairs_mean =  np.mean(subsib_Pairs, axis=0)
subsib_pairs_std =  np.std(subsib_Pairs, axis=0)

In [ ]:
subsib_conns_mean

In [ ]:
subsib_pairs_mean

In [ ]:
#subsib_connections/subsib_Pairs
#subsib_pairs_std =  np.std(subsib_connections/subsib_Pairs, axis=0)

In [ ]:
subsib_pairs_std

In [ ]:
ASD_Connections/N_Connections_total

In [ ]:
SIB_Connections/N_Connections_total

In [ ]:
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import matplotlib.ticker as mticker  
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 25})
fig, ax = plt.subplots(2,2,dpi=480, figsize=(16,12))

ax[0,0].plot(np.arange(6), ASD_Connections/N_Connections_total, marker="." , color="blue")
ax[0,0].plot(np.arange(6) + 0.05, SIB_Connections/N_Connections_total, marker=".", color="orange")
ax[0,0].errorbar(np.arange(6) + 0.05, 
                 subsib_conns_mean/N_Connections_total, 
                 yerr= np.nanstd(subsib_connections/N_Connections_total, axis=0), #subsib_conns_std/N_Connections_total,
                 marker=".", color="grey")

ax[0,0].grid(True)
ax[0,0].legend(loc="upper right", fontsize=20)
ax[0,0].set_ylabel("Connections Density", fontsize=25)
ax[0,0].set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax[0,0].set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax[0,0].tick_params(axis='y', labelsize=20)

ax[0,1].plot(np.arange(6), ASD_Connections/ASD_Pairs, marker="." , color="blue")
ax[0,1].plot(np.arange(6) + 0.05, SIB_Connections/SIB_Pairs, marker=".", color="orange")
ax[0,1].plot(np.arange(6) + 0.05, N_Connections_total/N_Pairs_total, marker=".", color="black")
ax[0,1].errorbar(np.arange(6) + 0.05, 
                 subsib_conns_mean/subsib_pairs_mean, 
                 yerr= np.nanstd(subsib_connections/subsib_Pairs, axis=0),#subsib_conns_std/subsib_pairs_mean,
                 marker=".", color="grey")

ax[0,1].grid(True)
ax[0,1].legend(loc="upper right", fontsize=20)
ax[0,1].set_ylabel("Connections Density 2", fontsize=25)
ax[0,1].set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax[0,1].set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax[0,1].tick_params(axis='y', labelsize=20)

ax[1,0].plot(np.arange(6), ASD_Pairs/N_Pairs_total, marker="." , color="blue")
ax[1,0].plot(np.arange(6) + 0.05, SIB_Pairs/N_Pairs_total, marker=".", color="orange")
ax[1,0].errorbar(np.arange(6) + 0.05, 
                 subsib_pairs_mean/N_Pairs_total, 
                 yerr= np.nanstd(subsib_Pairs/N_Pairs_total, axis=0),#subsib_pairs_std/N_Pairs_total,
                 marker=".", color="grey")

ax[1,0].grid(True)
ax[1,0].legend(loc="upper right", fontsize=20)
ax[1,0].set_ylabel("NPair Density", fontsize=25)
ax[1,0].set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax[1,0].set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax[1,0].tick_params(axis='y', labelsize=20)

norm = N_Connections_total/N_Pairs_total
ax[1,1].plot(np.arange(6), ASD_Connections/ASD_Pairs / norm , marker="." , color="blue")
ax[1,1].plot(np.arange(6) + 0.05, SIB_Connections/SIB_Pairs / norm, marker=".", color="orange")
ax[1,1].errorbar(np.arange(6) + 0.05, 
                 subsib_conns_mean/subsib_pairs_mean / norm, 
                 yerr= np.nanstd(subsib_connections/subsib_Pairs/norm, axis=0),
                 marker=".", color="grey")

ax[1,1].grid(True)
ax[1,1].legend(loc="upper right", fontsize=20)
ax[1,1].set_ylabel("Connections Density \nNormed by N_Pairs", fontsize=25)
ax[1,1].set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax[1,1].set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax[1,1].tick_params(axis='y', labelsize=20)
#plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%s x1000'))
plt.tight_layout()
#fig.savefig("figs/n_Fig_4.A.pdf")

In [ ]:
N_Connections_total

In [ ]:
ASD_Connections/subsib_conns_mean

In [ ]:
# significance of connection enrichment:
for asd, sibs in zip(ASD_Connections, subsib_connections.T):
    #print(asd, len(sibs))
    z,p = GetPermutationP(sibs, asd)
    print(p)

In [ ]:
# significance of connection enrichment:
for asd, sibs in zip(ASD_Connections, subsib_connections.T):
    #print(asd, len(sibs))
    #z,p = GetPermutationP(sibs, asd)
    #print(p)
    print(np.mean(sibs))

##### Combine multiple distances

In [ ]:
ASD_Connections

In [ ]:
subsib_connections

In [ ]:
ASD_Group1 = np.sum(ASD_Connections[:2])
ASD_Group2 = np.sum(ASD_Connections[:2])
Cont_Group1 =
Cont_Group2 = 

In [ ]:
ASD_Pairs

In [ ]:
ASD_Pairs/subsib_pairs_mean

In [ ]:
# significance of connection pairs:
for asd, sibs in zip(ASD_Pairs, subsib_Pairs.T):
    #print(asd, len(sibs))
    z,p = GetPermutationP(sibs, asd)
    print(p)

In [ ]:
(subsib_connections/subsib_Pairs / norm)

In [ ]:
(ASD_Connections/ASD_Pairs) / (subsib_conns_mean/subsib_pairs_mean)

In [ ]:
# significance of fc:
for asd, sibs in zip(ASD_Connections/ASD_Pairs / norm, (subsib_connections/subsib_Pairs / norm).T):
    #print(asd, len(sibs))
    z,p = GetPermutationP(sibs, asd)
    print(p)

In [ ]:
CaseGroup1 = np.sum(ASD_Connections[:2])/np.sum(N_Connections_total[:2])
CaseGroup2 = np.sum(ASD_Connections[2:])/np.sum(N_Connections_total[2:])
CtrlGroup1 = np.sum(subsib_connections[:, :2], axis=1)/np.sum(N_Connections_total[:2])
CtrlGroup2 = np.sum(subsib_connections[:, 2:], axis=1)/np.sum(N_Connections_total[2:])

# significance of fc:
z,p = GetPermutationP(CtrlGroup1, CaseGroup1)
print(p)

z,p = GetPermutationP(CtrlGroup2, CaseGroup2)
print(p)

In [ ]:
CaseGroup1,CaseGroup2

In [ ]:
norm
#norm = N_Connections_total/N_Pairs_total

In [ ]:
CaseGroup1 = np.sum(ASD_Connections[:2])/np.sum(ASD_Pairs[:2])/np.sum(N_Connections_total[:2]/N_Pairs_total[:2])
CaseGroup2 = np.sum(ASD_Connections[2:])/np.sum(ASD_Pairs[2:])/np.sum(N_Connections_total[2:]/N_Pairs_total[2:])
CtrlGroup1 = np.sum(subsib_connections[:, :2], axis=1)/np.sum(subsib_Pairs[:, :2], axis=1)/np.sum(N_Connections_total[:2]/N_Pairs_total[:2])
CtrlGroup2 = np.sum(subsib_connections[:, 2:], axis=1)/np.sum(subsib_Pairs[:, 2:], axis=1)/np.sum(N_Connections_total[2:]/N_Pairs_total[2:])

# significance of fc:
z,p = GetPermutationP(CtrlGroup1, CaseGroup1)
print(p)

z,p = GetPermutationP(CtrlGroup2, CaseGroup2)
print(p)

In [ ]:
CaseGroup1,CaseGroup2

In [ ]:
np.sum(ASD_Connections[:2])/np.sum(ASD_Pairs[:2])

In [ ]:
ASD_Connections/N_Connections_total

In [ ]:
subsib_conns_mean/N_Connections_total

In [ ]:
%matplotlib inline
import matplotlib.ticker as mticker  
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 25})
fig, ax = plt.subplots(dpi=480, figsize=(6,6))

ax.plot(np.arange(6), ASD_Connections/N_Connections_total, marker="o" , color="blue", label="ASD Circuits")
#ax.plot(np.arange(6) + 0.05, SIB_Connections/N_Connections_total, marker="o", color="orange", label="Sibling Circuits")
ax.errorbar(np.arange(6) + 0.05, 
                 subsib_conns_mean/N_Connections_total, 
                 yerr= np.nanstd(subsib_connections/N_Connections_total, axis=0), #subsib_conns_std/N_Connections_total,
                 marker="o", color="grey", label="Siblings Structures",)

ax.grid(True)
ax.legend(loc="bottom left", fontsize=17)
ax.set_ylabel("Connection Density", fontsize=25)
ax.set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.set_ylim(-0.015, 0.2)
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,6))
norm = N_Connections_total/N_Pairs_total
ax.plot(np.arange(6), ASD_Connections/ASD_Pairs / norm , marker="o" , color="blue", label="ASD Circuits")
#ax.plot(np.arange(6) + 0.05, SIB_Connections/SIB_Pairs / norm, marker="o", color="orange", label="Sibling Circuits")
ax.errorbar(np.arange(6) + 0.05, 
                 subsib_conns_mean/subsib_pairs_mean / norm, 
                 yerr= np.nanstd(subsib_connections/subsib_Pairs/norm, axis=0),
                 marker="o", color="grey", label="Siblings Structures")

ax.grid(True)
ax.legend(loc="upper left", fontsize=17)
ax.set_ylabel("Connection Likelihood Ratio", fontsize=25)
ax.set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
#plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%s x1000'))
#plt.tight_layout()
plt.show()

In [ ]:
ASD_Connections/ASD_Pairs / norm

In [ ]:
(subsib_conns_mean/subsib_pairs_mean / norm)

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,6))
ax.plot(np.arange(6), ASD_Pairs/N_Pairs_total, marker="." , color="blue", label="ASD Circuits")
#ax.plot(np.arange(6) + 0.05, SIB_Pairs/N_Pairs_total, marker=".", color="orange", label="Sibling Circuits")
ax.errorbar(np.arange(6) + 0.05, 
                 subsib_pairs_mean/N_Pairs_total, 
                 yerr= np.nanstd(subsib_Pairs/N_Pairs_total, axis=0),#subsib_pairs_std/N_Pairs_total,
                 marker=".", color="grey", label="SubSampled \nSiblings Circuits")

ax.grid(True)
ax.legend(loc="upper right", fontsize=20)
ax.set_ylabel("Structure Pairs Density", fontsize=25)
ax.set_xlabel(r"Distance range ($\times 10^3$ $\mu$m)", fontsize=20)
ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
plt.legend(fontsize=20)



# Doing with Sibling "circuits"

In [ ]:
def normtoUnit(x, xmin, xmax):
    return (x-xmin)/(xmax-xmin)

def searchFil(text, DIR):
    RES = []
    for file in os.listdir(DIR):
        if text in file:
            RES.append(file)
    return RES

def LoadSA3(fname, DIR, InfoMat, topL=100):
    fin = open(DIR+fname, 'rt')
    max_score, max_bias, max_STRs = 0, 0, []
    for i, l in enumerate(fin):
        if i > topL:
            break
        l = l.strip().split()
        bias = float(l[1])
        STRs = l[2].split(",")
        score = ScoreCircuit_SI_Joint(STRs, InfoMat)
        if score > max_score:
            max_score = score
            max_bias = bias
            max_STRs = STRs
    return max_score, max_bias, max_STRs

def GetData2(params, size, DIR, InfoMat):
    SCORES, CutBias, RealBias, STRS = [],[],[],[]
    for i, row in params.iterrows():
        fil = searchFil("keepN_{}-minbias_{}.txt".format(size, row["bias"]), DIR)[0]
        score, real_minbias, STRs = LoadSA3(fil, DIR, InfoMat)
        score = ScoreCircuit_SI_Joint(STRs, InfoMat)
        if score == 0:
            continue
        SCORES.append(score)
        CutBias.append(row["bias"])
        RealBias.append(real_minbias)
        STRS.append(STRs)
    return SCORES, CutBias, RealBias, STRS

def ExtractSibSTR(bias_cut, DIR, InfoMat):
    fil = searchFil("topN_121-keepN_46-minbias_{}.txt".format(bias_cut), DIR)[0]
    score, real_minbias, STRs = LoadSA3(fil, DIR, InfoMat) 
    return STRs

In [ ]:
biaslim_df = pd.read_csv(
    "../dat/Circuits/SA/biaslims2/biaslim.size.46.top17.txt", names=["size", "bias"])
InfoMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/InfoMat.Ipsi.csv", index_col=0)
adj_mat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/WeightMat.Ipsi.csv", index_col=0)

InfoMat_short = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/InfoMat.Ipsi.short.csv", index_col=0)

InfoMat_long = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/InfoMat.Ipsi.long.csv", index_col=0)

In [ ]:
biaslim_df.tail(5)

In [ ]:
ASD_STR_Bias = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", 
                           index_col="STR")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
ASD_Connections = []
ASD_Pairs = []
for i,v in enumerate(Distance_Cuts[:-1]):
    adj_mat_ = Cutted_AdjMat[i]
    dist_mat_ = Cutted_DistMat[i]
    adj_mat_asd = adj_mat_.loc[ASD_Circuits,ASD_Circuits]
    dist_mat_asd = dist_mat_.loc[ASD_Circuits,ASD_Circuits]
    Nconn = np.count_nonzero(adj_mat_asd)
    Npair = np.count_nonzero(dist_mat_asd)
    ASD_Connections.append(Nconn)
    ASD_Pairs.append(Npair)
ASD_Connections = np.array(ASD_Connections)
ASD_Pairs = np.array(ASD_Pairs)
print(ASD_STR_Bias.loc[ASD_Circuits, "EFFECT"].mean())

In [ ]:
# Get sibling circuits structures

SIB_SA_DIR = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/SubSib_ScoreInfo_Dec21/"
#SIB_SA_DIR = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/SubSib_Score_SI_Nov27_2023//"
#SIB_SA_DIR = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/SubSib_ScoreInfo_Sept_2023//"
bias_cut = 0.37
Sib_Cir_STRs = []
for i, file in enumerate(os.listdir(SIB_SA_DIR)):
    d = os.path.join(SIB_SA_DIR, file)
    if os.path.isdir(d):
        try:
            ASD_cont_Dir = SIB_SA_DIR + file + "/"
            Sib_CirSTR = ExtractSibSTR(bias_cut, ASD_cont_Dir, InfoMat)
            Sib_Cir_STRs.append(Sib_CirSTR)
        except:
            continue
    #if i > 10:
    #    break

In [ ]:
len(Sib_Cir_STRs)

In [ ]:
subsib_connections = []
subsib_Pairs = []
Sim_dir = "../dat/Unionize_bias/SubSampleSib//"
for i, _SibSTRs in enumerate(Sib_Cir_STRs):
    _SIB_Connections = []
    _SIB_Pairs = []
    for i,v in enumerate(Distance_Cuts[:-1]):
        adj_mat_ = Cutted_AdjMat[i]
        dist_mat_ = Cutted_DistMat[i]
        adj_mat_sib = adj_mat_.loc[_SibSTRs,_SibSTRs]
        dist_mat_sib = dist_mat_.loc[_SibSTRs,_SibSTRs]
        Nconn = np.count_nonzero(adj_mat_sib)
        Npair = np.count_nonzero(dist_mat_sib)
        _SIB_Connections.append(Nconn)
        _SIB_Pairs.append(Npair)
    subsib_connections.append(_SIB_Connections)
    subsib_Pairs.append(_SIB_Pairs)
subsib_connections = np.array(subsib_connections)
subsib_Pairs = np.array(subsib_Pairs)

In [ ]:
subsib_connections.shape

In [ ]:
subsib_conns_mean =  np.mean(subsib_connections, axis=0)
subsib_conns_std =  np.std(subsib_connections, axis=0)
subsib_pairs_mean =  np.mean(subsib_Pairs, axis=0)
subsib_pairs_std =  np.std(subsib_Pairs, axis=0)

In [ ]:
subsib_conns_mean

In [ ]:
subsib_pairs_mean

In [ ]:
subsib_pairs_std

In [ ]:
ASD_Connections/N_Connections_total

In [ ]:
SIB_Connections/N_Connections_total

In [ ]:
N_Connections_total

In [ ]:
ASD_Connections/subsib_conns_mean

In [ ]:
# significance of connection enrichment:
for asd, sibs in zip(ASD_Connections, subsib_connections.T):
    #print(asd, len(sibs))
    z,p = GetPermutationP(sibs, asd)
    print(p)

In [ ]:
# significance of connection enrichment:
for asd, sibs in zip(ASD_Connections, subsib_connections.T):
    #print(asd, len(sibs))
    #z,p = GetPermutationP(sibs, asd)
    #print(p)
    print(np.mean(sibs))

In [ ]:
ASD_Connections

In [ ]:
ASD_Pairs

In [ ]:
ASD_Pairs/subsib_pairs_mean

In [ ]:
# significance of connection pairs:
for asd, sibs in zip(ASD_Pairs, subsib_Pairs.T):
    #print(asd, len(sibs))
    z,p = GetPermutationP(sibs, asd)
    print(p)

In [ ]:
(ASD_Connections/ASD_Pairs) / (subsib_conns_mean/subsib_pairs_mean)

In [ ]:
#norm = N_Connections_total/N_Pairs_total

In [ ]:
# significance of fc:
norm = N_Connections_total/N_Pairs_total
for asd, sibs in zip(ASD_Connections/ASD_Pairs / norm, (subsib_connections/subsib_Pairs / norm).T):
    #print(asd, len(sibs))
    z,p = GetPermutationP(sibs, asd)
    print(p)

In [ ]:
split_idx = 3
CaseGroup1 = np.sum(ASD_Connections[:split_idx])/np.sum(N_Connections_total[:split_idx])
CaseGroup2 = np.sum(ASD_Connections[split_idx:])/np.sum(N_Connections_total[split_idx:])
CtrlGroup1 = np.sum(subsib_connections[:, :split_idx], axis=1)/np.sum(N_Connections_total[:split_idx])
CtrlGroup2 = np.sum(subsib_connections[:, split_idx:], axis=1)/np.sum(N_Connections_total[split_idx:])

# significance of fc:
z,p = GetPermutationP(CtrlGroup1, CaseGroup1)
print(p)

z,p = GetPermutationP(CtrlGroup2, CaseGroup2)
print(p)

In [ ]:
CaseGroup1,CaseGroup2

In [ ]:
norm

In [ ]:
split_idx = 3
CaseGroup1 = np.sum(ASD_Connections[:split_idx])/np.sum(ASD_Pairs[:split_idx])/np.sum(N_Connections_total[:split_idx]/N_Pairs_total[:split_idx])
CaseGroup2 = np.sum(ASD_Connections[split_idx:])/np.sum(ASD_Pairs[split_idx:])/np.sum(N_Connections_total[split_idx:]/N_Pairs_total[split_idx:])
CtrlGroup1 = np.sum(subsib_connections[:, :split_idx], axis=1)/np.sum(subsib_Pairs[:, :split_idx], axis=1)/np.sum(N_Connections_total[:split_idx]/N_Pairs_total[:split_idx])
CtrlGroup2 = np.sum(subsib_connections[:, split_idx:], axis=1)/np.sum(subsib_Pairs[:, split_idx:], axis=1)/np.sum(N_Connections_total[split_idx:]/N_Pairs_total[split_idx:])

# significance of fc:
z,p = GetPermutationP(CtrlGroup1, CaseGroup1)
print(p)

z,p = GetPermutationP(CtrlGroup2, CaseGroup2)
print(p)

In [ ]:
%matplotlib inline
import matplotlib.ticker as mticker  
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 25})
fig, ax = plt.subplots(dpi=480, figsize=(6,6))

ax.plot(np.arange(5), ASD_Connections/N_Connections_total, marker="o" , color="blue", label="ASD Circuits")
#ax.plot(np.arange(6) + 0.05, SIB_Connections/N_Connections_total, marker="o", color="orange", label="Sibling Circuits")
ax.errorbar(np.arange(5) + 0.05, 
                 subsib_conns_mean/N_Connections_total, 
                 yerr= np.nanstd(subsib_connections/N_Connections_total, axis=0), #subsib_conns_std/N_Connections_total,
                 marker="o", color="grey", label="Siblings Circuits",)

ax.grid(True)
ax.legend(loc="lower left", fontsize=17)
ax.set_ylabel("Connection Density", fontsize=25)
ax.set_xlabel(r"Distance range (mm)", fontsize=20)
#ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", r">4"], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
#ax.set_ylim(0, 0.17)
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,6))
norm = N_Connections_total/N_Pairs_total
ax.plot(np.arange(5), ASD_Connections/ASD_Pairs / norm , marker="o" , color="blue", label="ASD Circuits")
#ax.plot(np.arange(6) + 0.05, SIB_Connections/SIB_Pairs / norm, marker="o", color="orange", label="Sibling Circuits")
ax.errorbar(np.arange(5) + 0.05, 
                 subsib_conns_mean/subsib_pairs_mean / norm, 
                 yerr= np.nanstd(subsib_connections/subsib_Pairs/norm, axis=0),
                 marker="o", color="grey", label="Siblings Circuits")

ax.grid(True)
ax.legend(loc="upper left", fontsize=17)
ax.set_ylabel("Connection Likelihood Ratio", fontsize=25)
ax.set_xlabel(r"Distance range (mm)", fontsize=20)
#ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", r">4"], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
#plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%s x1000'))
#plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=480, figsize=(6,6))
ax.plot(np.arange(5), ASD_Pairs/N_Pairs_total, marker="." , color="blue", label="ASD Circuits")
#ax.plot(np.arange(6) + 0.05, SIB_Pairs/N_Pairs_total, marker=".", color="orange", label="Sibling Circuits")
ax.errorbar(np.arange(5) + 0.05, 
                 subsib_pairs_mean/N_Pairs_total, 
                 yerr= np.nanstd(subsib_Pairs/N_Pairs_total, axis=0),#subsib_pairs_std/N_Pairs_total,
                 marker=".", color="grey", label="SubSampled \nSiblings Circuits")

ax.grid(True)
ax.legend(loc="upper right", fontsize=20)
ax.set_ylabel("Structure Pairs Density", fontsize=25)
ax.set_xlabel(r"Distance range (mm)", fontsize=20)
#ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", "4-5", r">5"], fontsize=20)
ax.set_xticklabels(["", "0-1", "1-2", "2-3", "3-4", r">4"], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
plt.legend(fontsize=20)
